<a href="https://colab.research.google.com/github/minhhieu9800/MLE501/blob/main/Assignment_MLE501.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Giới thiệu

Họ tên: Nguyễn Minh Hiếu </br>
Lớp: MSE#11HCM </br>
Mã học viên: 22MSE23062 </br>
Email: hieu22mse23062@fsb.edu.vn

# Assignment

In [100]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from tabulate import tabulate

from keras.models import Sequential
from keras.layers import Dense
from keras.models import load_model
from keras.optimizers import Adam, RMSprop
from sklearn.model_selection import train_test_split

Load dữ liệu

In [53]:
url = 'https://minhhieu9800.github.io/MLE501/diabetes.csv'
df = pd.read_csv(url)

In [54]:
X = df.drop(['Outcome'], axis=1)
y = df['Outcome']

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2)

Xây dựng model

In [55]:
model = Sequential()
model.add(Dense(16, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.summary()

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=8, validation_data=(X_val, y_val))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 16)                144       
                                                                 
 dense_7 (Dense)             (None, 8)                 136       
                                                                 
 dense_8 (Dense)             (None, 1)                 9         
                                                                 
Total params: 289
Trainable params: 289
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
62/62 [==============================] - 2s 7ms/step - loss: 1.5768 - accuracy: 0.5601 - val_loss: 0.9109 - val_accuracy: 0.6016
Epoch 2/100
62/62 [==============================] - 0s 4ms/step - loss: 1.0408 - accuracy: 0.5967 - val_loss: 1.1059 - val_accuracy: 0.6260
Epoch 3/100
62/62 [===================

In [56]:
model.save("model.h5")

In [57]:
model = load_model("model.h5")

In [58]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Loss: ", loss)
print("Accuracy: ", accuracy)

5/5 [==============================] - 0s 5ms/step - loss: 0.6234 - accuracy: 0.6948
Loss:  0.6233633160591125
Accuracy:  0.6948052048683167


## Sử dụng Hyperparameter Tuning (Keras Tuner)

In [59]:
!pip install keras-tuner
import keras_tuner as kt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [60]:
# Tạm hàm build Hyper model
def build_model(hp): 
  model = keras.Sequential()

  X = hp.Int('units_input', min_value=8, max_value=64, step=8)
  model.add(Dense(units=X, input_dim=8, activation='relu'))

  for i in range(hp.Int('num_layers', 1, 4)):
    Y = hp.Int('units_' + str(i), min_value=8, max_value=64, step=8)
    model.add(Dense(units=Y, activation='relu'))

  model.add(Dense(1, activation='sigmoid'))

  lr = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
  model.compile(
        optimizer=Adam(learning_rate=lr),
        loss='binary_crossentropy',
        metrics=['accuracy'])
  
  return model

In [61]:
# Khởi tạo Tuner
tuner = kt.Hyperband (build_model, objective="val_accuracy", directory="tuner_dir_1", project_name="MLE501_Tuner_1")

In [62]:
tuner.search(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

In [63]:
# lấy best model
best_model = tuner.get_best_models(num_models=1)[0]

best_model.summary()

best_model.fit(X_train, y_train, epochs=100, batch_size=8, validation_data=(X_val, y_val))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                576       
                                                                 
 dense_1 (Dense)             (None, 24)                1560      
                                                                 
 dense_2 (Dense)             (None, 16)                400       
                                                                 
 dense_3 (Dense)             (None, 64)                1088      
                                                                 
 dense_4 (Dense)             (None, 16)                1040      
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 4,681
Trainable params: 4,681
Non-trainabl

In [64]:
best_model.save("best_model_hp.h5")

In [65]:
loss, accuracy = best_model.evaluate(X_test, y_test)
print("Best model using Hyperparameter Tuning")
print("Loss: ", loss)
print("Accuracy: ", accuracy)

model = load_model("model.h5")
loss, accuracy = model.evaluate(X_test, y_test)
print("Original model")
print("Loss: ", loss)
print("Accuracy: ", accuracy)

5/5 [==============================] - 0s 4ms/step - loss: 0.6350 - accuracy: 0.7013
Best model using Hyperparameter Tuning
Loss:  0.6350305676460266
Accuracy:  0.701298713684082
5/5 [==============================] - 0s 4ms/step - loss: 0.6234 - accuracy: 0.6948
Original model
Loss:  0.6233633160591125
Accuracy:  0.6948052048683167


### So với model ban đầu, model được cải tiến bằng Hyperparameter Tuning đã có sự cải thiện về độ chính xác. Sự thay đổi tạo ra từ việc các Hyperparameter đã được tối ưu có chọn lọc.




In [66]:
# Từ ban đầu model chỉ có 3 lớp với các thông số như sau:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 16)                144       
                                                                 
 dense_7 (Dense)             (None, 8)                 136       
                                                                 
 dense_8 (Dense)             (None, 1)                 9         
                                                                 
Total params: 289
Trainable params: 289
Non-trainable params: 0
_________________________________________________________________


In [67]:
# Model sau khi được tối ưu có 6 lớp với các thông số sau:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                576       
                                                                 
 dense_1 (Dense)             (None, 24)                1560      
                                                                 
 dense_2 (Dense)             (None, 16)                400       
                                                                 
 dense_3 (Dense)             (None, 64)                1088      
                                                                 
 dense_4 (Dense)             (None, 16)                1040      
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 4,681
Trainable params: 4,681
Non-trainabl

## Sử dụng Regularization (L2 Regularization)

In [68]:
from keras import regularizers

In [69]:
#áp dụng  với hệ số lambda là 0.01 cho cả 2 lớp Dense đầu tiên của kiến trúc mạng neural. 

model = Sequential()
model.add(Dense(16, input_dim=8, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(8, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=8, validation_data=(X_val, y_val))

Epoch 1/100
62/62 [==============================] - 2s 10ms/step - loss: 5.1253 - accuracy: 0.4582 - val_loss: 2.6120 - val_accuracy: 0.5285
Epoch 2/100
62/62 [==============================] - 0s 7ms/step - loss: 1.6584 - accuracy: 0.5642 - val_loss: 1.3408 - val_accuracy: 0.5691
Epoch 3/100
62/62 [==============================] - 0s 7ms/step - loss: 1.0960 - accuracy: 0.5784 - val_loss: 1.0640 - val_accuracy: 0.6260
Epoch 4/100
62/62 [==============================] - 0s 5ms/step - loss: 0.9855 - accuracy: 0.6273 - val_loss: 0.9708 - val_accuracy: 0.6585
Epoch 5/100
62/62 [==============================] - 0s 4ms/step - loss: 0.9293 - accuracy: 0.6232 - val_loss: 0.8660 - val_accuracy: 0.6667
Epoch 6/100
62/62 [==============================] - 0s 4ms/step - loss: 0.8735 - accuracy: 0.6517 - val_loss: 0.9272 - val_accuracy: 0.6504
Epoch 7/100
62/62 [==============================] - 0s 5ms/step - loss: 0.8720 - accuracy: 0.6436 - val_loss: 0.9117 - val_accuracy: 0.6829
Epoch 8/100


In [70]:
model.save("best_model_regularization.h5")

In [71]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Best model using Regularization (L2 Regularization)")
print("Loss: ", loss)
print("Accuracy: ", accuracy)

original = load_model("model.h5")
loss, accuracy = original.evaluate(X_test, y_test)
print("Original model")
print("Loss: ", loss)
print("Accuracy: ", accuracy)

5/5 [==============================] - 0s 4ms/step - loss: 0.6203 - accuracy: 0.7143
Best model using Regularization (L2 Regularization)
Loss:  0.6203097701072693
Accuracy:  0.7142857313156128
5/5 [==============================] - 0s 4ms/step - loss: 0.6234 - accuracy: 0.6948
Original model
Loss:  0.6233633160591125
Accuracy:  0.6948052048683167


### So với model ban đầu, model được cải tiến bằng Regularization đã có sự cải thiện về độ chính xác, cụ thể ở đây em dùng kỹ thuật L2 Regularization. Sự thay đổi tạo ra từ việc thêm một thành phần bình phương của các trọng số vào hàm mất mát. Việc này sẽ làm cho các trọng số nhỏ hơn, dẫn đến mô hình đơn giản hơn và tránh được việc bị overfitting.

## Sử dụng Optimization (Adam -> RMSprop)

In [72]:
model = Sequential()
model.add(Dense(16, input_dim=8, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(8, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=RMSprop(learning_rate=0.001), metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=8, validation_data=(X_val, y_val))

Epoch 1/100
62/62 [==============================] - 1s 7ms/step - loss: 15.1581 - accuracy: 0.3544 - val_loss: 5.5845 - val_accuracy: 0.4715
Epoch 2/100
62/62 [==============================] - 0s 4ms/step - loss: 3.1129 - accuracy: 0.5295 - val_loss: 1.4897 - val_accuracy: 0.5854
Epoch 3/100
62/62 [==============================] - 0s 5ms/step - loss: 1.1565 - accuracy: 0.5927 - val_loss: 1.0906 - val_accuracy: 0.6098
Epoch 4/100
62/62 [==============================] - 0s 4ms/step - loss: 1.0262 - accuracy: 0.5967 - val_loss: 0.9847 - val_accuracy: 0.6748
Epoch 5/100
62/62 [==============================] - 0s 6ms/step - loss: 0.9598 - accuracy: 0.6110 - val_loss: 0.9427 - val_accuracy: 0.6423
Epoch 6/100
62/62 [==============================] - 0s 7ms/step - loss: 0.9146 - accuracy: 0.6130 - val_loss: 0.9021 - val_accuracy: 0.6504
Epoch 7/100
62/62 [==============================] - 0s 6ms/step - loss: 0.8755 - accuracy: 0.6232 - val_loss: 0.8616 - val_accuracy: 0.6585
Epoch 8/100


In [73]:
model.save("best_model_optimization.h5")

In [74]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Best model using Optimization (Adam -> RMSprop)")
print("Loss: ", loss)
print("Accuracy: ", accuracy)

original = load_model("model.h5")
loss, accuracy = original.evaluate(X_test, y_test)
print("Original model")
print("Loss: ", loss)
print("Accuracy: ", accuracy)

5/5 [==============================] - 0s 4ms/step - loss: 0.6272 - accuracy: 0.7208
Best model using Optimization (Adam -> RMSprop)
Loss:  0.6271921396255493
Accuracy:  0.7207792401313782
5/5 [==============================] - 0s 4ms/step - loss: 0.6234 - accuracy: 0.6948
Original model
Loss:  0.6233633160591125
Accuracy:  0.6948052048683167


### So với model ban đầu, model được cải tiến bằng Regularization đã có sự cải thiện về độ chính xác, cụ thể ở đây em thay đổi optimizer từ Adam sang RMSprop. Thuật toán tối ưu mô hình sẽ thay đổi cách tính toán gradient và cập nhật trọng số trong quá trình huấn luyện. Cụ thể, hai thuật toán này sử dụng hai phương pháp khác nhau để tính toán gradient và cập nhật trọng số, do đó việc thay đổi optimizer sẽ có tác động đến hiệu suất của mô hình.

## Kết hợp cả 3 kỹ thuật

In [75]:
# Tạm hàm build Hyper model
def build_best_model(hp): 
  model = keras.Sequential()

  X = hp.Int('units_input', min_value=8, max_value=64, step=8)
  model.add(Dense(units=X, input_dim=8, activation='relu', kernel_regularizer=regularizers.l2(0.01)))

  for i in range(hp.Int('num_layers', 1, 4)):
    Y = hp.Int('units_' + str(i), min_value=8, max_value=64, step=8)
    model.add(Dense(units=Y, activation='relu', kernel_regularizer=regularizers.l2(0.01)))

  model.add(Dense(1, activation='sigmoid'))

  # Tìm kiếm optimizer
  optimizer = hp.Choice('optimizer', values=['adam', 'rmsprop'])

  # Tìm kiếm learning_rate
  lr = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  if optimizer == 'adam':
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=lr), metrics=['accuracy'])
  else:
    model.compile(loss='binary_crossentropy', optimizer=RMSprop(learning_rate=lr), metrics=['accuracy'])
  
  return model

In [76]:
# Khởi tạo Tuner
tuner = kt.Hyperband (build_best_model, objective="val_accuracy", directory="tuner_dir_2", project_name="MLE501_Tuner_2")

In [77]:
tuner.search(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

In [78]:
# Lấy ra các tham số
best_hps = tuner.get_best_hyperparameters()[0]

print("units_input: ", best_hps.get('units_input'))
print("num_layers: ", best_hps.get('num_layers'))
print("optimizer: ", best_hps.get('optimizer'))
print("learning_rate: ", best_hps.get('learning_rate'))

units_input:  16
num_layers:  4
optimizer:  adam
learning_rate:  0.01


In [ ]:
# lấy super model
best_model = tuner.get_best_models(num_models=1)[0]

best_model.summary()

best_model.fit(X_train, y_train, epochs=100, batch_size=8, validation_data=(X_val, y_val))

In [80]:
best_model.save("best_model_super.h5")

In [97]:
modelOriginal = load_model("model.h5")
modelHp = load_model("best_model_hp.h5")
modelRe = load_model("best_model_regularization.h5")
modelOp = load_model("best_model_optimization.h5")
modelSuper = load_model("best_model_super.h5")

lossOriginal, accuracyOriginal = modelOriginal.evaluate(X_test, y_test)
lossHp, accuracyHp = modelHp.evaluate(X_test, y_test)
lossRe, accuracyRe = modelRe.evaluate(X_test, y_test)
lossOp, accuracyOp = modelOp.evaluate(X_test, y_test)
lossSuper, accuracySuper = modelSuper.evaluate(X_test, y_test)

5/5 [==============================] - 0s 4ms/step - loss: 0.5387 - accuracy: 0.7403


In [102]:
# create the DataFrame
df = pd.DataFrame({'': ['Original','Hyperparameter Tuning (Keras Tuner)','Regularization (L2 Regularization)','Optimization (Adam -> RMSprop)','Combined 3 ways'],
                   'Loss': [lossOriginal, lossHp, lossRe, lossOp, lossSuper],
                   'Accuracy': [accuracyOriginal, accuracyHp, accuracyRe, accuracyOp, accuracySuper]})

# print the table using tabulate
print(tabulate(df, headers='keys', tablefmt='pipe', floatfmt='.4f'))

|    |                                     |   Loss |   Accuracy |
|---:|:------------------------------------|-------:|-----------:|
|  0 | Original                            | 0.6234 |     0.6948 |
|  1 | Hyperparameter Tuning (Keras Tuner) | 0.6350 |     0.7013 |
|  2 | Regularization (L2 Regularization)  | 0.6203 |     0.7143 |
|  3 | Optimization (Adam -> RMSprop)      | 0.6272 |     0.7208 |
|  4 | Combined 3 ways                     | 0.5387 |     0.7403 |
